In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from featexp import get_trend_stats

In [10]:
data = pd.read_csv('train.csv',encoding='big5', low_memory=False)
data.shape

(100000, 132)

In [11]:
data_test = pd.read_csv('test.csv',encoding='big5', low_memory=False)
data_test.shape

(150000, 131)

## Check over NaN & space

## Categorical columns 

In [23]:
object_data = data.select_dtypes(include='object').columns
len(object_data)

90

###   1)  ordinal feature (replace NaN with 0)

In [24]:
def order_features(df):
    order_mapping = {'低':1,'中':2,'中高':3,'高':4}
    col=['AGE','APC_1ST_AGE','INSD_1ST_AGE','RFM_R','REBUY_TIMES_CNT','LIFE_CNT']
    for i in col:
        df[i] = df[i].map(order_mapping)
        df[i] = df[i].fillna(0)
    return df

In [25]:
data_object_1 = order_features(data.copy())

In [26]:
data_test = order_features(data_test)

###   2-1) nominal feature_Y/N (replace NaN with 2)

In [27]:
def Y_N(df):
    count = 0
    transform = {'Y':1,'N':0}
    for i in df.columns:
        if re.match(r'IF|FINANCETOOLS|X_|IM_IS|LAST|^[A-Z].*IND$', i):
            df[i] = df[i].map(transform)
            df[i] = df[i].fillna(2)
            count += 1  
    print("number of Y/N columns ：", count)         
    
    try:
        df['Y1'] = df['Y1'].map(transform)  # Using 'try' to make sure not to raise error when fitting test data 
    except:
        pass

    return df

In [28]:
data_object_2 = Y_N(data_object_1)

number of Y/N columns ： 79


In [29]:
data_test = Y_N(data_test)

number of Y/N columns ： 79


###   2-2) nominal feature_others (make NaN become a kind of category)

In [30]:
def OHE(df):
    col = df.select_dtypes(include='object').columns
    col = col.append(pd.Index(["MARRIAGE_CD"])) # !!! It's a categorical column in float data type !!! 
    print('The remaining categorical columns：', len(col), "\n", col)
    
    c3 = {}
    for c in col:
        c3[c] = 'ohe_' + c
        #df[c] = df[c].fillna("NaN") # to make "NaN" also become a kind of category
        
    df = df.fillna("NaN")  #if use Fill_NA function
    df = pd.get_dummies(df, columns=col, drop_first=True, prefix=c3)
    
    print('Shape：', df.shape)
    return df

In [31]:
data_object_3 = OHE(data_object_2)

The remaining categorical columns： 5 
 Index(['GENDER', 'CHARGE_CITY_CD', 'CONTACT_CITY_CD', 'CUST_9_SEGMENTS_CD',
       'MARRIAGE_CD'],
      dtype='object')
Shape： (100000, 153)


In [32]:
data_test = OHE(data_test)

The remaining categorical columns： 5 
 Index(['GENDER', 'CHARGE_CITY_CD', 'CONTACT_CITY_CD', 'CUST_9_SEGMENTS_CD',
       'MARRIAGE_CD'],
      dtype='object')
Shape： (150000, 152)


In [33]:
def Fill_NaN(df, method):
    imr = Imputer(missing_values='NaN', strategy=method, axis=0).fit(df.values)
    imputed_data = imr.transform(df.values)
    
    #turn numpy.ndarray back to dataframe
    col={}
    for j,c in enumerate(df.columns):
        col[c] = imputed_data[:, j]
        
    df = pd.DataFrame(col)
    return df

In [34]:
data_object_final = Fill_NaN(data_object_3, 'mean')
data_test_final = Fill_NaN(data_test, 'mean')

C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\doggy\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


## Numeric columns 

In [14]:
check_col = []
for i in data_object_3.columns:
    if len(data_object_3[i].unique()) > 5: #因order_features的unique=5
        check_col.append(i)
len(check_col)

37

In [15]:
check_col.remove('CUS_ID')

In [65]:
set(data.columns) - set(object_data) - set(check_col) #The remaining unchecked columns

{'APC_CNT',
 'CUS_ID',
 'EDUCATION_CD',
 'IM_CNT',
 'L1YR_B_ISSUE_CNT',
 'MARRIAGE_CD'}

In [82]:
others = list(set(data.columns) - set(object_data) - set(check_col))
others.remove('MARRIAGE_CD')
for i in others:
    if data_object_3[i].isnull().any() == True:
        print(i)
        data_object_3[i] = data_object_3[i].fillna(data_object_3[i].dropna().mean())

EDUCATION_CD


In [96]:
for i in others:
    if data_test[i].isnull().any() == True:
        print(i)
        data_test[i] = data_test[i].fillna(data_test[i].dropna().mean())

EDUCATION_CD


## 1) outliers

In [87]:
def outlier_check_IQR(check_col, col_checked, df):
    for i in check_col:
        col_value = df[i].dropna()
        Percentile = np.percentile(col_value.unique(), [0,25,50,75,100])
        IQR = Percentile[3] - Percentile[1] #四分位距
        UpLimit = Percentile[3] + IQR*1.5 # 約 +2.698 std
        DownLimit = Percentile[1] - IQR*1.5

        if len(col_value[col_value > UpLimit]) > 0 or len(col_value[col_value < DownLimit]) > 0:
            y_1 = sum(df.iloc[col_value[col_value > UpLimit]]["Y1"]) + \
                  sum(df.iloc[col_value[col_value < DownLimit]]["Y1"])
            print('col: %s, Over UpLimit: %d, Under DownLimit: %d, Y=1: %d' % \
                  (i, len(col_value[col_value > UpLimit]),len(col_value[col_value < DownLimit]),y_1))

            col_checked.append(i)

    print('\n')
    print('col_checked :' , len(col_checked))
    print('No outliers: ', set(check_col)-set(col_checked))

In [88]:
col_checked = []
outlier_check_IQR(check_col, col_checked, data_object_3)
# outlier中沒有y=1者  -->不做特別處理

col: INSD_CNT, Over UpLimit: 1, Under DownLimit: 0, Y=1: 0
col: ANNUAL_PREMIUM_AMT, Over UpLimit: 2901, Under DownLimit: 0, Y=1: 0
col: ANNUAL_INCOME_AMT, Over UpLimit: 3323, Under DownLimit: 0, Y=1: 0
col: TOOL_VISIT_1YEAR_CNT, Over UpLimit: 5, Under DownLimit: 0, Y=1: 0
col: DIEBENEFIT_AMT, Over UpLimit: 1560, Under DownLimit: 0, Y=1: 0
col: DIEACCIDENT_AMT, Over UpLimit: 2732, Under DownLimit: 0, Y=1: 0
col: POLICY_VALUE_AMT, Over UpLimit: 865, Under DownLimit: 0, Y=1: 0
col: ANNUITY_AMT, Over UpLimit: 97, Under DownLimit: 0, Y=1: 0
col: EXPIRATION_AMT, Over UpLimit: 25, Under DownLimit: 0, Y=1: 0
col: OUTPATIENT_SURGERY_AMT, Over UpLimit: 39, Under DownLimit: 0, Y=1: 0
col: INPATIENT_SURGERY_AMT, Over UpLimit: 11, Under DownLimit: 0, Y=1: 0
col: PAY_LIMIT_MED_MISC_AMT, Over UpLimit: 71, Under DownLimit: 0, Y=1: 0
col: FIRST_CANCER_AMT, Over UpLimit: 43, Under DownLimit: 0, Y=1: 0
col: ILL_ACCELERATION_AMT, Over UpLimit: 252, Under DownLimit: 0, Y=1: 0
col: ILL_ADDITIONAL_AMT, Over 

## 2) NAN

In [95]:
col_continuous, col_discrete = [], []
for i in check_col:
    #連續型數值(結尾AMT者、BMI、APC_1ST_YEARDIF、TERMINATION_RATE)
    if re.match(r'\w+AMT$|BMI|APC_1ST_YEARDIF|TERMINATION_RATE', i):
        col_continuous.append(i)
        
    #離散型數值
    else:
        col_discrete.append(i)

print('col_continuous:%d, col_discrete:%d, columns need to check:%d' % \
      (len(col_continuous), len(col_discrete), len(col_continuous)+ len(col_discrete)))

col_continuous:21, col_discrete:15, columns need to check:36


###   2-1) col_continuous #21

In [90]:
def NaN_check(col, col_nan, df):
    for i in col:
        if df[i].isnull().any() == True:
            col_nan.append(i)
            nan_count = len(df[i][df[i].isnull() == True])
            y1 = sum(df['Y1'][df[i].isnull() == True])
            print('col: %s, NaNs: %d, Y=1: %d, P(NaNs|Y=1): %.4f' % (i, nan_count, y1, y1/2000))

    print('\n')
    print('col_nan: ',len(col_nan))
    print('No NANs: ', set(col)-set(col_nan))

In [91]:
col_continuous_nan = []
NaN_check(col_continuous, col_continuous_nan, data_object_3)
# APC_1ST_YEARDIF、ANNUAL_PREMIUM_AMT、ANNUAL_INCOME_AMT、 BMI、TERMINATION_RATE --> 用 KNN Regression 填補
# 其他 --> 用 mean 填補

col: APC_1ST_YEARDIF, NaNs: 43282, Y=1: 387, P(NaNs|Y=1): 0.1935
col: ANNUAL_PREMIUM_AMT, NaNs: 62445, Y=1: 604, P(NaNs|Y=1): 0.3020
col: ANNUAL_INCOME_AMT, NaNs: 39201, Y=1: 407, P(NaNs|Y=1): 0.2035
col: BMI, NaNs: 16645, Y=1: 427, P(NaNs|Y=1): 0.2135
col: TERMINATION_RATE, NaNs: 43282, Y=1: 387, P(NaNs|Y=1): 0.1935
col: DIEBENEFIT_AMT, NaNs: 27540, Y=1: 333, P(NaNs|Y=1): 0.1665
col: DIEACCIDENT_AMT, NaNs: 27540, Y=1: 333, P(NaNs|Y=1): 0.1665
col: POLICY_VALUE_AMT, NaNs: 27540, Y=1: 333, P(NaNs|Y=1): 0.1665
col: ANNUITY_AMT, NaNs: 27540, Y=1: 333, P(NaNs|Y=1): 0.1665
col: EXPIRATION_AMT, NaNs: 27540, Y=1: 333, P(NaNs|Y=1): 0.1665
col: ACCIDENT_HOSPITAL_REC_AMT, NaNs: 27540, Y=1: 333, P(NaNs|Y=1): 0.1665
col: DISEASES_HOSPITAL_REC_AMT, NaNs: 27540, Y=1: 333, P(NaNs|Y=1): 0.1665
col: OUTPATIENT_SURGERY_AMT, NaNs: 27540, Y=1: 333, P(NaNs|Y=1): 0.1665
col: INPATIENT_SURGERY_AMT, NaNs: 27540, Y=1: 333, P(NaNs|Y=1): 0.1665
col: PAY_LIMIT_MED_MISC_AMT, NaNs: 27540, Y=1: 333, P(NaNs|Y=1): 0.1

#### KNN填補：https://mp.weixin.qq.com/s?__biz=MzIzNDk0ODIxNg==&mid=2247483697&idx=1&sn=769babea47a36fcd2eb27e9b5586b227&chksm=e8efd226df985b3049d98471c463705161e8c2412e28e5f5d09dc6e68c675875799bc56d9621&mpshare=1&scene=1&srcid=1019sGnZIzqL4buSlqvXwydT#rd

In [92]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

def find_no_nan_columns(df):
    remain_nan_cols = df.isnull().any() [df.isnull().any()== True].index.tolist()
    all_cols =  df.columns.tolist()
    all_cols.remove('CUS_ID')
    try:
        all_cols.remove('Y1')
    except:
        pass
    
    no_nan_cols = list(set(all_cols) - set(remain_nan_cols))
    return no_nan_cols

def knn_train_test(target_col, df, no_nan_cols):
    train_y = df[target_col][df[target_col].isnull() == False] #目標欄位下的非NAN值
    train_x = df[df[target_col].isnull() == False].loc[:,no_nan_cols] #行列都沒有NAN值者
    test_x = df[df[target_col].isnull() == True].loc[:,no_nan_cols] #目標欄位下含NAN值者的其餘非NAN列值
    return train_y, train_x, test_x

def knn_missing_filled(train_x, train_y, test_x, k = 3, dispersed = True):
    if dispersed:
        clf = KNeighborsClassifier(n_neighbors = k, weights = "distance")
    else:
        clf = KNeighborsRegressor(n_neighbors = k, weights = "distance")
    
    clf.fit(train_x, train_y)
    return test_x.index, clf.predict(test_x)

def filled_pred_to_data(target_columns, df , n=3, data_type=True):
    for i in target_columns:
        no_nan_cols = find_no_nan_columns(df)
        train_y, train_x, test_x =  knn_train_test(i, df, no_nan_cols)
        index, pred = knn_missing_filled( train_x, train_y, test_x, k = n, dispersed = data_type)
        df.loc[index, i] = pred
        print(i," complete")
    return df

In [93]:
target_cols = ['APC_1ST_YEARDIF','ANNUAL_PREMIUM_AMT','ANNUAL_INCOME_AMT','BMI','TERMINATION_RATE']

In [94]:
data_train_knn = filled_pred_to_data(target_cols, data_object_3, n=5, data_type=False)

APC_1ST_YEARDIF  complete
ANNUAL_PREMIUM_AMT  complete
ANNUAL_INCOME_AMT  complete
BMI  complete
TERMINATION_RATE  complete


In [97]:
data_test_knn = filled_pred_to_data(target_cols, data_test, n=5, data_type=False)

APC_1ST_YEARDIF  complete
ANNUAL_PREMIUM_AMT  complete
ANNUAL_INCOME_AMT  complete
BMI  complete
TERMINATION_RATE  complete


In [99]:
others = [x for x in col_continuous_nan if x not in target_cols]
for i in others:
        data_train_knn[i] = data_train_knn[i].fillna(data_train_knn[i].dropna().mean())
        data_test_knn[i] = data_test_knn[i].fillna(data_test_knn[i].dropna().mean())

###   2-2) col_discrete #15

In [ ]:
'''
# Outlier_check_plot
for i in col_discrete:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.title('train')
    sns.boxplot(y=i, data=data_train_knn.dropna())
    
    plt.subplot(1,2,2)
    plt.title('test')
    sns.boxplot(y=i, data=data_test_knn.dropna())
    plt.show()
'''

In [102]:
col_discrete_nan = []
NaN_check(col_discrete, col_discrete_nan, data_object_3)
# Notice：L1YR_C_CNT

col: OCCUPATION_CLASS_CD, NaNs: 3960, Y=1: 286, P(NaNs|Y=1): 0.1430
col: LEVEL, NaNs: 43305, Y=1: 390, P(NaNs|Y=1): 0.1950
col: RFM_M_LEVEL, NaNs: 43282, Y=1: 387, P(NaNs|Y=1): 0.1935
col: L1YR_C_CNT, NaNs: 87936, Y=1: 1436, P(NaNs|Y=1): 0.7180
col: INSD_LAST_YEARDIF_CNT, NaNs: 171, Y=1: 64, P(NaNs|Y=1): 0.0320


col_nan:  5
No NANs:  {'CHANNEL_B_POL_CNT', 'BANK_NUMBER_CNT', 'INSD_CNT', 'CLC_CUR_NUM', 'CHANNEL_A_POL_CNT', 'AG_NOW_CNT', 'TOOL_VISIT_1YEAR_CNT', 'L1YR_A_ISSUE_CNT', 'AG_CNT', 'LIFE_INSD_CNT'}


In [104]:
# check 'L1YR_C_CNT' -not nan values
a = data_train_knn['L1YR_C_CNT'][data_train_knn['L1YR_C_CNT'].isnull() == False]
print('Y1=1: ', sorted(a[data_train_knn["Y1"]==1].unique()))
print('Y1=0: ', sorted(a[data_train_knn["Y1"]==0].unique()))
print('test: ', sorted(data_test_knn['L1YR_C_CNT'][data_test_knn['L1YR_C_CNT'].isnull() == False].unique()))

Y1=1:  [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 29.0]
Y1=0:  [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 21.0, 22.0, 23.0, 24.0, 25.0, 27.0, 29.0, 30.0, 31.0, 35.0, 37.0, 41.0]
test:  [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 29.0, 47.0]


In [115]:
# check 'L1YR_C_CNT' -create new column(L1YR_C_CNT <= 15 = 1 else = 0)
new_col = 'L1YR_C_CNT_over15'
old_col = 'L1YR_C_CNT'
data_train_knn[new_col] = 0
index = data_train_knn[old_col][data_train_knn[old_col] <= 15].index.tolist()
data_train_knn[new_col][index] = 1

data_test_knn[new_col]=0
index = data_test_knn[old_col][data_test_knn[old_col] <= 15].index.tolist()
data_test_knn[new_col][index] = 1

C:\Users\doggy\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\doggy\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [117]:
print('origin_corr: %.4f | new_corr: %.4f' % \
      (data_train_knn['L1YR_C_CNT'].corr(data_train_knn['Y1']),\data_train_knn['L1YR_C_CNT_over15'].corr(data_train_knn['Y1'])))

origin_corr: 0.0091 | new_corr: 0.0708


In [118]:
target_cols = ['L1YR_C_CNT', 'LEVEL', 'RFM_M_LEVEL']

In [119]:
data_train_knn = filled_pred_to_data(target_cols, data_train_knn, n=5, data_type=True)

L1YR_C_CNT  complete
LEVEL  complete
RFM_M_LEVEL  complete


In [120]:
data_test_knn = filled_pred_to_data(target_cols, data_test_knn, n=5, data_type=True)

L1YR_C_CNT  complete
LEVEL  complete
RFM_M_LEVEL  complete


In [122]:
others = [x for x in col_discrete_nan if x not in target_cols]
for i in others:
        data_train_knn[i] = data_train_knn[i].fillna(data_train_knn[i].dropna().mean())
        data_test_knn[i] = data_test_knn[i].fillna(data_test_knn[i].dropna().mean())

In [123]:
print('Final train shape: ',data_train_knn.shape, '/','Final test shape: ',data_test_knn.shape)

Final train shape:  (100000, 154) / Final test shape:  (150000, 153)


In [126]:
data_train_knn.to_csv('train_reprocess_knn_v2.csv',index=False,encoding='big5')
data_test_knn.to_csv('test_reprocess_knn_v2.csv',index=False,encoding='big5')

## -----------------------------------------------------------------------------------------------------------------

### New Columns

In [62]:
train = pd.read_csv('train_reprocess_knn_v2.csv',encoding='big5', low_memory=False)
test = pd.read_csv('test_reprocess_knn_v2.csv',encoding='big5', low_memory=False)

In [63]:
len(set(train['CUS_ID']).difference(test['CUS_ID']))

94207

In [64]:
def corr_compare_and_drop(df, target_cols, new_col, drop_col):
    #刪除相關係數小於新增欄位者
    print(new_col, df[new_col].corr(df['Y1']))
    for i in target_cols:
        print(i, df[i].corr(df['Y1']))
        if abs(df[i].corr(df['Y1'])) < abs(df[new_col].corr(df['Y1'])):
            drop_col.append(i)
    return drop_col

def relation_print(x1_col, x2_col):
    print(x1_col +'： %.5f' % train[x1_col].corr(train['Y1']),'\n',
          x2_col +'： %.5f' % train[x2_col].corr(train['Y1']))
    
def plot_count(x, data):
    sns.countplot(x= x, data=data)
    plt.title("cor: %.4f" % train[x].corr(train['Y1']))

In [65]:
print("train： %s, test： %s" % (train.shape, test.shape))

train： (100000, 154), test： (150000, 153)


In [66]:
# 1. 目前"壽險保單"持有有效主約件數
col_ind = [i for i in train.columns if re.match(r'IF_ISSUE_._IND', i)]
train['ISSUE_IND_SUM'] = train[col_ind].sum(axis=1)
test['ISSUE_IND_SUM'] = test[col_ind].sum(axis=1)

drop_col = []
corr_compare_and_drop(train, col_ind, 'ISSUE_IND_SUM', drop_col)

train = train.drop(drop_col, axis=1)
test = test.drop(drop_col, axis=1)
print("train： %s, test： %s" % (train.shape, test.shape))

ISSUE_IND_SUM 0.11223029052368377
IF_ISSUE_A_IND 0.008663317877697632
IF_ISSUE_B_IND 0.020617739022457327
IF_ISSUE_C_IND 0.027731887229823667
IF_ISSUE_D_IND 0.04078368311190528
IF_ISSUE_E_IND 0.0023859095550657346
IF_ISSUE_F_IND 0.01065180753647559
IF_ISSUE_G_IND 0.03342295287227516
IF_ISSUE_H_IND 0.005114230284501293
IF_ISSUE_I_IND 0.103050244565999
IF_ISSUE_J_IND 0.07464773664538958
IF_ISSUE_K_IND 0.004107279945545776
IF_ISSUE_L_IND 0.01991533943111337
IF_ISSUE_M_IND -4.108476040975764e-18
IF_ISSUE_N_IND 0.041581561746040406
IF_ISSUE_O_IND -0.0015310220811699633
IF_ISSUE_P_IND 0.05069155057720931
IF_ISSUE_Q_IND 0.09783821318614783


['IF_ISSUE_A_IND',
 'IF_ISSUE_B_IND',
 'IF_ISSUE_C_IND',
 'IF_ISSUE_D_IND',
 'IF_ISSUE_E_IND',
 'IF_ISSUE_F_IND',
 'IF_ISSUE_G_IND',
 'IF_ISSUE_H_IND',
 'IF_ISSUE_I_IND',
 'IF_ISSUE_J_IND',
 'IF_ISSUE_K_IND',
 'IF_ISSUE_L_IND',
 'IF_ISSUE_M_IND',
 'IF_ISSUE_N_IND',
 'IF_ISSUE_O_IND',
 'IF_ISSUE_P_IND',
 'IF_ISSUE_Q_IND']

In [67]:
# 2. 目前"壽險保單"持有有效附約件數
col_ind = [i for i in train.columns if re.match(r'IF_ADD_._IND', i)]
train['ISSUE_ADD_SUM'] = train[col_ind].sum(axis=1)
test['ISSUE_ADD_SUM'] = test[col_ind].sum(axis=1)

drop_col = []
corr_compare_and_drop(train, col_ind, 'ISSUE_ADD_SUM', drop_col)

train = train.drop(drop_col, axis=1)
test = test.drop(drop_col, axis=1)
print("train： %s, test： %s" % (train.shape, test.shape))

ISSUE_ADD_SUM 0.0884988904409944
IF_ADD_F_IND 0.0401263641862706
IF_ADD_L_IND 0.0940249989927532
IF_ADD_Q_IND 0.09406823613616394
IF_ADD_G_IND 0.004714228362330166
IF_ADD_R_IND 0.06660611264631974


['IF_ADD_F_IND', 'IF_ADD_G_IND', 'IF_ADD_R_IND']

In [107]:
# 3. 年收入 > 年繳化保費 ? 
a = train['ANNUAL_INCOME_AMT'].rank() - train['ANNUAL_PREMIUM_AMT'].rank()
train['ANNUAL_INCOME_minus_PREMIUM'] = a.apply(lambda x: 1 if x > 0 else 0)

b = test['ANNUAL_INCOME_AMT'].rank() - test['ANNUAL_PREMIUM_AMT'].rank()
test['ANNUAL_INCOME_minus_PREMIUM'] = b.apply(lambda x: 1 if x > 0 else 0)

print('ANNUAL_INCOME_minus_PREMIUM: ',train['ANNUAL_INCOME_minus_PREMIUM'].corr(train['Y1']))
relation_print('ANNUAL_INCOME_AMT','ANNUAL_PREMIUM_AMT')

ANNUAL_INCOME_minus_PREMIUM:  0.027471924531535803
ANNUAL_INCOME_AMT： 0.00894 
 ANNUAL_PREMIUM_AMT： 0.00335


In [47]:
train = train.drop(['ANNUAL_INCOME_AMT','ANNUAL_PREMIUM_AMT'], axis=1)
test = test.drop(['ANNUAL_INCOME_AMT','ANNUAL_PREMIUM_AMT'], axis=1)
print("train： %s, test： %s" % (train.shape, test.shape))

train： (100000, 135), test： (150000, 134)


In [69]:
# 4. 是否持有特定商品(A-D)
col_ind = [i for i in train.columns if re.match(r'IM_IS_._IND', i)]
train['IM_IS_SUM_IND'] = train[col_ind].sum(axis=1)
test['IM_IS_SUM_IND'] = test[col_ind].sum(axis=1)

drop_col = []
corr_compare_and_drop(train, col_ind, 'IM_IS_SUM_IND', drop_col)

train = train.drop(drop_col, axis=1)
test = test.drop(drop_col, axis=1)
print("train： %s, test： %s" % (train.shape, test.shape))

IM_IS_SUM_IND 0.07178148299138282
IM_IS_A_IND 0.012886988755502378
IM_IS_B_IND 0.05929916132453058
IM_IS_C_IND 0.040783574215572806
IM_IS_D_IND 0.04803408184577384
train： (100000, 158), test： (150000, 157)


In [70]:
train.to_csv('train_reprocess_knn_v2_newcolumns.csv',index=False,encoding='big5')
test.to_csv('test_reprocess_knn_v2_newcolumns.csv',index=False,encoding='big5')